In [48]:
import pyroute2
import socket
ipr = pyroute2.IPRoute()

ipr.addr("add", index=ipr.link_lookup(ifname="wlan0")[0], address="10.0.0.2", prefixlen=24)

()

In [71]:
import pyroute2
import socket
ipr = pyroute2.IPRoute()
route = {
'dst': '10.0.0.0/24',
'oif': ipr.link_lookup(ifname='wlan0')[0],
'scope': 253,
'prefsrc': '10.0.0.2'
}
#ipr.route("add", **route)
#ipr.route("add", oif=ipr.link_lookup(ifname="wlan0")[0], dst="10.0.0.0/24", prefsrc="10.0.0.1")
ipr.route("del", oif=ipr.link_lookup(ifname="wlan0")[0], dst="10.0.0.0/24", scope=253)

({'error': 0,
  'attrs': [('NLMSGERR_ATTR_UNUSED', None)],
  'header': {'length': 36,
   'type': 2,
   'flags': 256,
   'sequence_number': 257,
   'pid': 2541037863,
   'error': None,
   'target': 'localhost',
   'stats': Stats(qsize=0, delta=0, delay=0)},
  'event': 'NLMSG_ERROR'},)

In [97]:

for dump in ipr.route("dump", oif=ipr.link_lookup(ifname="eth0")[0]):
    if 'dst_len' in dump.keys():
        print(dump)


{'family': 2, 'dst_len': 0, 'src_len': 0, 'tos': 0, 'table': 254, 'proto': 3, 'scope': 0, 'type': 1, 'flags': 0, 'attrs': [('RTA_TABLE', 254), ('RTA_GATEWAY', '192.168.1.1'), ('RTA_OIF', 2)], 'header': {'length': 52, 'type': 24, 'flags': 2, 'sequence_number': 305, 'pid': 2541037863, 'error': None, 'target': 'localhost', 'stats': Stats(qsize=0, delta=0, delay=0)}, 'event': 'RTM_NEWROUTE'}
{'family': 2, 'dst_len': 24, 'src_len': 0, 'tos': 0, 'table': 254, 'proto': 2, 'scope': 253, 'type': 1, 'flags': 0, 'attrs': [('RTA_TABLE', 254), ('RTA_DST', '192.168.1.0'), ('RTA_PREFSRC', '192.168.1.39'), ('RTA_OIF', 2)], 'header': {'length': 60, 'type': 24, 'flags': 2, 'sequence_number': 305, 'pid': 2541037863, 'error': None, 'target': 'localhost', 'stats': Stats(qsize=0, delta=0, delay=0)}, 'event': 'RTM_NEWROUTE'}
{'family': 2, 'dst_len': 32, 'src_len': 0, 'tos': 0, 'table': 255, 'proto': 2, 'scope': 254, 'type': 2, 'flags': 0, 'attrs': [('RTA_TABLE', 255), ('RTA_DST', '192.168.1.39'), ('RTA_PREFS

In [98]:
from sysctl_utilities import *

In [2]:
from pyroute2_utilities import *
import scapy.all
import constants.mac_templates as mac_templates
random_mac = scapy.all.RandMAC(mac_templates.random_mac_template())
print(random_mac)
set_host_data(ifname="wlan0", ip="10.0.0.2", mac=random_mac, ttl=21, netmask=24, mtu=1300)

74:51:ba:ec:37:01
2024-08-13 22:14:35,943 [INFO] pyroute2_utilities [change_mac]: wlan0 is busy shutting down the interface
2024-08-13 22:14:36,061 [INFO] pyroute2_utilities [change_mac]: starting up wlan0


In [40]:
import pyroute2
import socket
import constants.sysctl_params as sysctl_params
import shell_utilities

ipr = pyroute2.IPRoute()
ifname="wlan0"

simple_dump = {}
ipv4s = []
ipv6s = []
for addr_dump in ipr.addr("dump", index=ipr.link_lookup(ifname=ifname)[0]):
    if addr_dump['family'] == socket.AF_INET:
        temp_ipv4 = addr_dump.get_attr("IFA_ADDRESS") + "/" + str(addr_dump['prefixlen'])
        ipv4s.append(temp_ipv4)
    if addr_dump['family'] == socket.AF_INET6:
        temp_ipv6 = addr_dump.get_attr("IFA_ADDRESS") + "/" + str(addr_dump['prefixlen'])
        ipv6s.append(temp_ipv6)
        
link_dump = ipr.poll(ipr.link, "dump", ifname=ifname)[0]
mac = link_dump.get_attr("IFLA_ADDRESS")
mtu = link_dump.get_attr("IFLA_MTU")

ttl = shell_utilities.get_sysctl_param(sysctl_params.IPV4_DEFAULT_TTL)

print(ipv4s)
print(ipv6s)
print(mac)

simple_dump.update({"ipv4" : ipv4s})
simple_dump.update({"ipv6" : ipv6s})
simple_dump.update({"mac" : mac})
simple_dump.update({"mtu" : mtu})
simple_dump.update({"ttl" : ttl})


print(simple_dump)

print("192.168.1.52/24" in simple_dump['ipv4'])

['192.168.1.52/24']
['fe80::369a:40c4:c83:5291/64']
64:b4:73:aa:89:8d
{'ipv4': ['192.168.1.52/24'], 'ipv6': ['fe80::369a:40c4:c83:5291/64'], 'mac': '64:b4:73:aa:89:8d', 'mtu': 1500, 'ttl': '64'}
True


In [12]:
tuple(x for i in range(20))

NameError: name 'x' is not defined

In [4]:
import pyroute2
import socket
import pytest
import sys
import os
from test_pyroute2_utilities import *
sys.path.insert(0, os.path.abspath('../..'))

from reface2.utilities.pyroute2_utilities import *
def combine_tuples(*arrays):
    combined = []
    for combination in itertools.product(*arrays):
        first_entries = tuple([t[0] for t in combination])
        second_entries = [t[1] for t in combination]
        iter_result = first_entries + ((second_entries,))
        combined.append(iter_result)
    return combined

# Example usage
array1 = [('param1', 0), ('param2', 1)]
array2 = [('x1', 2), ('x3', 3)]
array3 = [('y1', 4), ('y2', 5)]

result = combine_tuples(array1, array2, array3)
print(params['ifname'])

[('mgQl9z2', 19), ('YEltQ2z', 19), ('eyzPIGf', 19), ('L3yTrDY', 19), ('AHyYRfv', 19), ('jaVYtlT', 19), ('CTitwKp', 19), ('oQykOhs', 19), ('mzdDFSN', 19), ('NuslVfl', 19), 'wlan0']


## from functools import wraps

def atomic(*get_state_funcs):
    """
    A decorator that ensures the atomicity of the decorated function.
    The function takes as an argument a variable number of callables `get_state_funcs`
    that retrieve the current state of the target attributes.

    If the function fails (returns a non-zero error code), the state of all the 
    attributes should remain unchanged.

    Parameters:
    *get_state_funcs (callables): Functions that retrieve the current state of 
                                  the target attributes.

    Usage:
        @atomic(get_attrX, get_attrY)
        def some_function():
            ...
    """
    
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            # Get the initial state of all target attributes
            initial_states = [get_func() for get_func in get_state_funcs]
            print(f"initial state : {initial_states}")
            # Execute the function and get the result (assuming errno is returned)
            errno=0
            try:
                errno = func(*args, **kwargs)
            except:
                final_states = [get_func() for get_func in get_state_funcs]
                print(f"after state : {final_states}")
                assert initial_states == final_states, (
                    "Atomicity violated! "
                    f"Initial states: {initial_states}, Final states: {final_states}"
                )
            # If function failed, ensure the state hasn't changed
            if errno:
                final_states = [get_func() for get_func in get_state_funcs]
                print(f"after state : {final_states}")
                assert initial_states == final_states, (
                    "Atomicity violated! "
                    f"Initial states: {initial_states}, Final states: {final_states}"
                )
            
            return errno
            
        return wrapper
    
    return decorator


from pyroute2_utilities import*

get_mac = lambda : ipr.link("dump", ifname="wlan0")[0].get_attr("IFLA_ADDRESS")

@atomic(get_mac)
def set_mac(ifname, mac):
    try:
        change_mac(ifname, mac)
    except Exception as e:
        ipr.link("set", ifname=ifname, address= "b4:8c:9d:ad:4d:c9")
        raise e
    
set_mac("wlan0", "FF:FF:FF:FF:FF:FF")

In [25]:
dic={
    "mac" : ipr.link("dump", ifname="wlan0")[0].get_attr("IFLA_ADDRESS")
}
dic["mac"]

'b4:8c:9d:ad:4d:c9'

In [27]:
[1] + [2] + [1]

[1, 2, 1]

In [48]:
2**8

256

In [61]:
try: 
    ipr.link("set", index = ipr.link_lookup(ifname="wlan0"), address='00:00:00:00:00:00')
except Exception as e:
    print(e)

(99, 'Cannot assign requested address')


In [60]:
from ipaddress import *
IPv4Address(2**32)

AddressValueError: 4294967296 (>= 2**32) is not permitted as an IPv4 address

In [16]:
ifname = 'fS1TS3w'
ip = IPv4Address('58.194.214.152')
mac = 'FF:FF:FF:FF:FF:FF'
netmask = 308
ttl = 196
mtu = -92
expected_exception_codes = [19, 99, 22]

from pyroute2_utilities import *
set_host_data(ip)


SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (1282231569.py, line 1)